In [1]:

#from TestDiff.ipynb import Crystal
import numpy as np
import ctypes


#Wrapper

#path to the dll
lib = ctypes.CDLL("E:/Arxius varis/PhD/2nd_year/Code/Diffraction_pattern_simulation_E_V/Library/difftool_1.1/diffTools.dll")
#lib = ctypes.cdll.LoadLibrary("E:/Arxius varis/PhD/2nd_year/Code/Diffraction_pattern_simulation_E_V/Library/difftool/diffTools.dll")


CrystalHandle = ctypes.POINTER(ctypes.c_char)
c_int_array = np.ctypeslib.ndpointer(dtype=np.int32, ndim=1, flags='C_CONTIGUOUS')
c_int_array_2 = np.ctypeslib.ndpointer(dtype=np.int32, ndim=2, flags='C_CONTIGUOUS')


lib.createCrystal.argtypes = [ctypes.c_char_p]
lib.createCrystal.restype = CrystalHandle

lib.calc_d.argtypes = [CrystalHandle, ctypes.c_bool, ctypes.c_float]
lib.calc_d.restypes = ctypes.c_int

lib.FindZA.argtypes = [CrystalHandle, ctypes.c_float, ctypes.c_float, ctypes.c_float, ctypes.c_float]
lib.FindZA.restype = ctypes.c_int

lib.GetZA.argtypes = [CrystalHandle, ctypes.c_int, c_int_array]
lib.GetZA.restype = None

lib.destroyCrystal.argtypes = [CrystalHandle]
lib.destroyCrystal.restype = None

lib.full_info_from_reflections.argtypes= [CrystalHandle, ctypes.c_bool, ctypes.c_float, c_int_array_2, c_int_array, c_int_array]
lib.full_info_from_reflections.restype = None



class Crystal:
    
    def __init__(self,name):
        self.instance = lib.createCrystal(name)
    
    def __del__(self):
        lib.destroyCrystal(self.instance)
    
    def getZA(self,N):
        n = ctypes.c_int(N)
        hkl = np.empty(3, dtype=np.int32)
        lib.GetZA(self.instance,n,hkl)
        return hkl
    
    def Diff(self,flag,D):
        min_d = ctypes.c_float(D)
        flagd=ctypes.c_bool(flag)
        N = lib.calc_d(self.instance,flagd,min_d)
        return N
    
    def FindZA(self,D1,D2,ANG,TOL):
        d1=ctypes.c_float(D1)
        d2=ctypes.c_float(D2)
        ang=ctypes.c_float(ANG)
        tol=ctypes.c_float(TOL)
        self.n = lib.FindZA(self.instance,d1,d2,ang,tol)
        return self.n
    
    def full_info_from_reflections(self, flag, D):
        min_d = ctypes.c_float(D)
        flagd=ctypes.c_bool(flag) 
        N = lib.calc_d(self.instance,flagd,min_d)
        
        hkl_s=np.empty((N,3), dtype=np.int32)
        distances=np.empty(N, dtype=np.int32)
        F_factors=np.empty(N, dtype=np.int32)
        
        lib.full_info_from_reflections(self.instance,flagd, min_d,hkl_s,distances, F_factors)
        
        return hkl_s, distances, F_factors
   
import ctypes
import numpy as np
import matplotlib.pyplot as plt
import skimage.measure as measure
import cv2
import torch
import stemtool
import hyperspy.api as hs
from Diffraction_simulation_ZA_finding import Crystal
import time

def Find_percentage_of_thresholded(FFT_image, threshold):
    FFT_image[FFT_image<=threshold]=0
    percentage_of_tresholded=np.count_nonzero(FFT_image.ravel())/len(FFT_image.ravel())
    return percentage_of_tresholded

def Threshold_given_percentage(FFT_image, percentage):
    y_pixs,x_pixs=np.shape(FFT_image)
    n_int_pixs=int(round(percentage*y_pixs*x_pixs))
    FFT_ravel=np.sort(np.ravel(FFT_image))[::-1]
    threshold=FFT_ravel[n_int_pixs]
    return threshold

def mpfit_Distance(FFT_image,FOV):
    mpfit_model=[[-2.87175127e-11],
                     [ 8.11320079e-09],
                     [-8.18658056e-07],
                     [ 3.33222163e-05],
                     [-2.02745223e-04],
                     [-2.26140649e-02],
                     [ 5.95346985e-01],
                     [-7.69005862e-01]]
    # without the averages
    mpfit_model_c=[[-3.46636981e-11],
                   [ 1.00423053e-08],
                   [-1.06223267e-06],
                   [ 4.84860471e-05],
                   [-6.82330526e-04],
                   [-1.58450088e-02],
                   [ 5.79540436e-01],
                   [-1.10510783e+00]]
    #set the working limits of the model
    if FOV >=30:
        mpfit_dist=np.array([40])
    else:
        
        fov_vals=np.array([FOV**7,FOV**6,FOV**5,FOV**4,FOV**3,FOV**2,FOV**1,1])
        mpfit_dist=np.e**np.dot(fov_vals,mpfit_model)
        mpfit_dist=np.e**np.dot(fov_vals,mpfit_model_c)
     
    #Adjustments depending on the sizze of the image
    if np.shape(FFT_image)[0]==2048:
        mpfit_dist=mpfit_dist*1.30
    elif np.shape(FFT_image)[0]<256:
        mpfit_dist=mpfit_dist*1.55     
    elif np.shape(FFT_image)[0]==256:
        mpfit_dist=mpfit_dist*1.55     
    elif np.shape(FFT_image)[0]==1024:
        mpfit_dist=mpfit_dist*1.05
    elif np.shape(FFT_image)[0]==512:
        mpfit_dist=mpfit_dist*1.15
    else:
        mpfit_dist=mpfit_dist*1.15
        
    return mpfit_dist[0]

def FFT_threshold(FOV):
    FFT_thresh_model=[[-1.01291174e-11],
                          [ 2.88297492e-09],
                          [-3.01778444e-07],
                          [ 1.44327587e-05],
                          [-3.23378617e-04],
                          [ 3.61163733e-03],
                          [-3.72515413e-02],
                          [-1.96361805e-01]]
    # without the averages
    FFT_thresh_model_c=[[ 1.54099057e-12],
                        [-6.56354380e-10],
                        [ 1.05878669e-07],
                        [-8.09680716e-06],
                        [ 2.96148198e-04],
                        [-4.30807411e-03],
                        [ 1.81389577e-03],
                        [-2.45698182e-01]]
    #set the working limits of the model
    if FOV >=80:
        FFT_thresh=np.array([0.6])
    else:
                
        fov_vals=np.array([FOV**7,FOV**6,FOV**5,FOV**4,FOV**3,FOV**2,FOV**1,1])
        FFT_thresh=np.e**np.dot(fov_vals,FFT_thresh_model)
        FFT_thresh=np.e**np.dot(fov_vals,FFT_thresh_model_c)
      
    
    return FFT_thresh[0]

def FFT_percentage(FFT_image,FOV):
    FFT_perc_model=[[-3.00411834e-11],
                        [ 1.17313244e-08],
                        [-1.81232383e-06],
                        [ 1.40635117e-04],
                        [-5.76020214e-03],
                        [ 1.20704617e-01],
                        [-1.20113823e+00],
                        [-2.14024711e+00]]
    # without the averages
    FFT_perc_model_c=[[ 1.38602821e-11],
                      [-2.46874956e-09],
                      [-1.63526870e-08],
                      [ 2.67725990e-05],
                      [-1.91230990e-03],
                      [ 5.28789844e-02],
                      [-6.40863899e-01],
                      [-3.71037505e+00]]
    #set the working limits of the model
    if FOV >=110:
        FFT_perc=np.array([0.00025])  #In case it is too much for higher FOVs, just delete this and keep the FFT_perc_model for all ranges
    # elif FOV <3:
    #     FFT_perc=np.array([0.01])
    else:
        
        
        fov_vals=np.array([FOV**7,FOV**6,FOV**5,FOV**4,FOV**3,FOV**2,FOV**1,1])
        FFT_perc=np.e**np.dot(fov_vals,FFT_perc_model)
        FFT_perc=np.e**np.dot(fov_vals,FFT_perc_model_c)        
        
        if FOV <4.5:
            FFT_perc=FFT_perc*(10**(np.log(128/np.shape(FFT_image)[0])/np.log(4)))
        elif FOV >=4.5 and FOV <=20 :
            FFT_perc=FFT_perc*(10**(np.log(512/np.shape(FFT_image)[0])/np.log(4)))
        else:
            FFT_perc=FFT_perc*(10**(np.log(2048/np.shape(FFT_image)[0])/np.log(4)))
    
    #Adjustments depending on the sizze of the image
    if np.shape(FFT_image)[0]<256:
        FFT_perc=FFT_perc*0.25
    elif np.shape(FFT_image)[0]==256:
        FFT_perc=FFT_perc*0.45  
    elif np.shape(FFT_image)[0]==512:
        FFT_perc=FFT_perc*0.55
    elif np.shape(FFT_image)[0]==1024:
        FFT_perc=FFT_perc*0.80    
    else:
        FFT_perc=FFT_perc*0.80
        
    return FFT_perc[0]

def FFT_hyperparams(FFT_image,FOV):
    #Return, in order, the mpfit dist, the threshold, and the percentage
    
    mpfit_dist=mpfit_Distance(FFT_image,FOV)
    FFT_thresh=FFT_threshold(FOV)
    FFT_perc=FFT_percentage(FFT_image,FOV)
    
    return mpfit_dist,FFT_thresh,FFT_perc



def FFT_calibration(hyperspy_2D_signal):
    fft_shifted = hyperspy_2D_signal.fft(shift=True)
    
    FFT_calibration=fft_shifted.axes_manager['x'].scale
    FFT_pixels=fft_shifted.axes_manager['x'].size
    FFT_units=fft_shifted.axes_manager['x'].units
    
    return FFT_calibration,FFT_pixels,FFT_units
    

def Spot_coord_To_d_spacing_indiv(coords, FFT_calibration, FFT_pixels):
    
    (y_max, x_max)=coords

    FFT_distance_point_x=np.abs(x_max-int(FFT_pixels/2))*FFT_calibration
    FFT_distance_point_y=np.abs(y_max-int(FFT_pixels/2))*FFT_calibration
    
    FFT_distance_total=np.sqrt(FFT_distance_point_x**2+FFT_distance_point_y**2)
    
    
    d_spacing_spot=1/FFT_distance_total
    
    return d_spacing_spot



def Spot_coord_To_d_spacing_vect(coord_vects, FFT_calibration, FFT_pixels):
    y_vects=coord_vects[:,0]    
    x_vects=coord_vects[:,1] 

    FFT_distance_point_x=np.abs(x_vects-int(FFT_pixels/2))*FFT_calibration
    FFT_distance_point_y=np.abs(y_vects-int(FFT_pixels/2))*FFT_calibration
    
    FFT_distance_total=np.sqrt(FFT_distance_point_x**2+FFT_distance_point_y**2)
    
    
    d_spacing_spot=1/FFT_distance_total
    
    return d_spacing_spot


def Spot_coord_To_Angles_to_X_indiv(coords,FFT_pixels):
    
    (y_max, x_max)=coords
    
    cont_dist=x_max-int(FFT_pixels/2)
    opp_dist=int(FFT_pixels/2)-y_max
    
    angles_to_X=np.arctan2(opp_dist,cont_dist)*180/np.pi
    
    return angles_to_X


def Spot_coord_To_Angles_to_X_vect(coord_vects,FFT_pixels):
    y_vects=coord_vects[:,0]    
    x_vects=coord_vects[:,1] 
    
    cont_dist=x_vects-int(FFT_pixels/2)
    opp_dist=int(FFT_pixels/2)-y_vects
    
    angles_to_X=np.arctan2(opp_dist,cont_dist)*180/np.pi
    
    return angles_to_X


def Ensure_Center_Diff(distances_array, angles_to_x_array):
    
    #define hyperparameter, that should not be very modifyable, as it is the 
    #maximum interplanar distance to consider, in nm, let us say d_int > 1-1.5nm
    #as no plane should be  bigger than 1-1.5nm. Let us say 1.5 to include 1/2 indices 
    
    #what we do is remove these spots from the array, as they should not be considered
    #either in angle or distance, so this already solves unwanted spots at a very small frequency or high dist.
    
    distances_array_c=distances_array[distances_array<=1.5]
    angles_to_x_array_c=angles_to_x_array[distances_array<=1.5]
   
    return distances_array_c, angles_to_x_array_c



#Hyperparameters
gauss_blur_filter_size=(5,5)  #size of smoothing filter, go to line to change sigma
downscaling_factor=1 #for trials, n factor of downsampling size of image
FFT_thresholding=0.5  #value above which the pixels are kept
st_distance=30 #distance parameter in the Stem tool method
FFT_thresholdingG=0.6 #value above which the pixels are kept, in the gaussian filtered FFT
window_size=128  #window size of the sliding windows
np.random.seed(int(time.time()))

#dm3 loading, and calibration extraction
imagedm3=hs.load(r'E:\Arxius varis\PhD\2nd_year\Code\trial_images\dm3_atomic_resolution\GeQW2.dm3')
meta1=imagedm3.metadata
meta2=imagedm3.original_metadata.export('parameters')



x_calibration=imagedm3.axes_manager['x'].scale
y_calibration=imagedm3.axes_manager['y'].scale

x_pixels=imagedm3.axes_manager['x'].size
y_pixels=imagedm3.axes_manager['y'].size

x_units=imagedm3.axes_manager['x'].units
y_units=imagedm3.axes_manager['y'].units


#FFT calibration

FFT_calibration,FFT_pixels,FFT_units=FFT_calibration(imagedm3)


imagearray=np.asarray(imagedm3)
image=imagearray

plt.imshow(image, cmap=plt.cm.gray, vmin=image.min(), vmax=image.max())
plt.show()
#First standarisation of the image for filtering/blurring it with gaussian filter
image_st=(image-np.min(image))/np.max(image-np.min(image))



#Application of Gaussian filter for denoising


denoised_image=cv2.GaussianBlur(image_st, gauss_blur_filter_size, 1)


#Second standarisation of the image after filtering/blurring it with gaussian filter

image_st=(denoised_image-np.min(denoised_image))/np.max(denoised_image-np.min(denoised_image))

#Print histogram



#For sake of evaluation, better work with an image with less pixels, as only the consecutive pixel evaluation would take
#approximately 6 hours to run for a big region of 250.000 pixels in total.

#Then downsample the image and upsample it posteriorly 
#We select a max pooling method to keep track of the brighter elements and this way keep a higher contrast


ds_image=measure.block_reduce(image_st, block_size=tuple(np.int32(downscaling_factor*np.ones(len(np.shape(image_st))))), func=np.mean, cval=0)

#and standarise it again to ensure 0-1 values

ds_image_st=(ds_image-np.min(ds_image))/np.max(ds_image-np.min(ds_image))


# take the fft of the image
fft_image_w_background = np.fft.fftshift(np.log(np.fft.fft2(ds_image_st)))
fft_abs_image_background = np.abs(fft_image_w_background)

# apply the filter
fft_abs_image_background2=np.copy(fft_abs_image_background)
fft_abs_image_backgroundc=np.copy(fft_abs_image_background)


fft_abs_image_backgroundc=(fft_abs_image_backgroundc-np.min(fft_abs_image_backgroundc))/np.max(fft_abs_image_backgroundc-np.min(fft_abs_image_backgroundc))


fft_abs_image_background2=(fft_abs_image_background2-np.min(fft_abs_image_background2))/np.max(fft_abs_image_background2-np.min(fft_abs_image_background2))


fft_abs_image_background2=cv2.GaussianBlur(fft_abs_image_background2, gauss_blur_filter_size, 1)
fft_abs_image_background2=(fft_abs_image_background2-np.min(fft_abs_image_background2))/np.max(fft_abs_image_background2-np.min(fft_abs_image_background2))


#trial with original FFT
#fft_abs_image_background2=fft_abs_image_backgroundc

#Automatic hyperparameter finding
fov=np.shape(fft_abs_image_background2)[0]*y_calibration
print('fov',fov)
st_distance,_,FFT_perc=FFT_hyperparams(fft_abs_image_background2,fov)
print('mpfit',st_distance,'perc',FFT_perc )
FFT_thresholdingG=Threshold_given_percentage(fft_abs_image_background2, FFT_perc)
print('fft_threhs',FFT_thresholdingG)


print('STEM Tool based method (2D Gaussians)')

center_difractogram=(int(FFT_pixels/2), int(FFT_pixels/2))

print('ST gaussian FFT')
twodfit_blur=stemtool.afit.peaks_vis(fft_abs_image_background2, dist=st_distance, thresh=FFT_thresholdingG, imsize=(15, 15))

d_distances=Spot_coord_To_d_spacing_vect(twodfit_blur, FFT_calibration, FFT_pixels)

angles_to_x=Spot_coord_To_Angles_to_X_vect(twodfit_blur,FFT_pixels)

print(d_distances)

print(angles_to_x)



    

refined_distances, refined_angles_to_x=Ensure_Center_Diff(d_distances, angles_to_x)

#set the values of distances in angstroms
refined_distances=refined_distances*10    

#prepare the distances to be analysed

#start by taking a random spot
distance1=np.random.choice(refined_distances)
#starting with the first spot at approx 90 degrees from x axis could be nice as well

indexdist1=np.argmin(np.abs(refined_distances-distance1)) 
distances2=np.delete(refined_distances, indexdist1)
angles=np.abs(refined_angles_to_x-refined_angles_to_x[indexdist1])
angles=np.delete(angles, indexdist1)


#Watch out in case angles need to be between 0 and 180 to limit this, although it should work as well.
#Maybe take the center of diffraction out after computing to avoid confusions, as it is not a diffracted spot




#Diffraction simulation of a given cell


Ge_cell = Crystal(b'E:/Arxius varis/PhD/2nd_year/Code/unit_cells/Ge.uce')

min_d=0.5    #minimum interplanar distance computed in the diffraction
forbidden = False  #Include (True) or not (False) the forbbiden reflections

n = Ge_cell.Diff(forbidden,min_d)
print("Calculated ",n ,"reflections")

tol=0.05   #tolerance: how different from theoretical values the previous values can be to get good output

ZA_list=[]

for d2, angle in zip(distances2,angles):
    print("Between distance 1 = {:.3f} A and distance 2 = {:.3f} A, there are {:.3f} degrees".format(distance1, d2, angle))


    n = Ge_cell.FindZA(distance1,d2,angle,tol)
    print("Found ",n,"possible Zone Axes")
    
    #Include or not the possibility to output high index axes
    high_index=False
    
    if n==0:
        ZA=None
    else:
        if high_index==True:
            ZA=Ge_cell.getZA(0)
        else:
            for index in range(n):
                ZA=Ge_cell.getZA(index)
                if np.sum(np.abs(ZA))< 10: #if sum of indices is higher than 10 is already high indexes
                    break
                else:
                    ZA=None
                    continue
        
    
    print(ZA)
    
    if type(ZA)==type(None):
        ZA_list.append(ZA)
    else:
        ZA_list.append(np.sort(np.abs(ZA)))


ZA_list_index=[index for index in ZA_list if type(index)!=type(None)]
ZA_arr_index=np.asarray(ZA_list_index)


temp_array_ZA = np.ascontiguousarray(ZA_arr_index).view(np.dtype((np.void, ZA_arr_index.dtype.itemsize * ZA_arr_index.shape[1])))
_, idx,counts = np.unique(temp_array_ZA, return_index=True, return_counts=True)

ZA_unique = np.unique(temp_array_ZA).view(ZA_arr_index.dtype).reshape(-1, ZA_arr_index.shape[1])


ZA_final=ZA_unique[np.argmax(counts)]
print('Found ZA:', ZA_final)



hkl_s, distances, F_factors=Ge_cell.full_info_from_reflections(forbidden,min_d)

print(hkl_s)
print(distances)
print(F_factors)



AttributeError: function 'full_info_from_reflections' not found